# Gestão Salarial

## Importações

In [13]:
import pandas as pd
import numpy as np
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

## Implementação

In [2]:
# ---------------------------
# CONFIGURAÇÃO
# ---------------------------
DATA_INICIO = "25/01/2026"   # dd/mm/aaaa
VALOR_BASE = 5              # semana 1
INCREMENTO = 5              # +5 por semana
TOTAL_SEMANAS = 52


# ---------------------------
# LÓGICA
# ---------------------------
data_inicio = datetime.strptime(DATA_INICIO, "%d/%m/%Y")

semanas = []
data_semana = data_inicio

for n_semana in range(1, TOTAL_SEMANAS + 1):
    valor_semana = VALOR_BASE + (n_semana - 1) * INCREMENTO
    
    semanas.append({
        "numero_semana": n_semana,
        "data_inicio_semana": data_semana.date(),
        "mes": data_semana.strftime("%Y-%m"),
        "valor_semana": float(valor_semana),
    })
    
    data_semana += relativedelta(days=7)

df_semanas = pd.DataFrame(semanas)

# Soma por mês (com base no mês do INÍCIO da semana)
df_mensal = (
    df_semanas
    .groupby("mes", as_index=False)["valor_semana"]
    .sum()
    .rename(columns={"valor_semana": "investimento_mes"})
)

df_mensal

,mes,investimento_mes
0,2026-01,5.0
1,2026-02,70.0
2,2026-03,200.0
3,2026-04,250.0
4,2026-05,425.0
5,2026-06,430.0
6,2026-07,510.0
7,2026-08,750.0
8,2026-09,690.0
9,2026-10,770.0


In [4]:
# =========================
# CONFIG
# =========================
DIVIDAS_CSV = "data/dividas.csv"
OUTPUT_CSV  = "data/tabela_unificada_cartao_desafio.csv"

# Desafio 52 semanas
DATA_INICIO_52S = "25/01/2026"   # dd/mm/aaaa
BASE_52S = 5.0
INCREMENTO_52S = 5.0
TOTAL_SEMANAS = 52

# Ajuste manual do cartão
MES_AJUSTE_CARTAO = "2026-02"
VALOR_CARTAO_FEVEREIRO = 1089.08


# =========================
# HELPERS
# =========================
def parse_number(x) -> float:
    """Aceita '1.234,56', '1234,56', '1234.56', 'R$ 1.234,56' etc."""
    if pd.isna(x):
        return 0.0
    if isinstance(x, (int, float)):
        return float(x)

    s = str(x).strip()
    if not s:
        return 0.0

    s = s.replace("R$", "").replace(" ", "")

    if "," in s and "." in s:
        # decide decimal pelo último separador
        if s.rfind(",") > s.rfind("."):
            s = s.replace(".", "").replace(",", ".")   # pt-BR
        else:
            s = s.replace(",", "")                     # en-US
    elif "," in s:
        s = s.replace(".", "").replace(",", ".")       # pt-BR simples
    else:
        # pode ser 1.234.567 (milhar com ponto)
        parts = s.split(".")
        if len(parts) > 2:
            s = "".join(parts[:-1]) + "." + parts[-1]

    try:
        return float(s)
    except ValueError:
        return 0.0


def month_start(d: date) -> date:
    return date(d.year, d.month, 1)

def add_months(d: date, n: int) -> date:
    return (d + relativedelta(months=n)).replace(day=1)

def month_key(d: date) -> str:
    return f"{d.year:04d}-{d.month:02d}"

def build_month_range(start_month: date, end_month: date):
    months = []
    cur = start_month
    while cur <= end_month:
        months.append(cur)
        cur = add_months(cur, 1)
    return months


# =========================
# 1) CARTÃO: gerar cronograma mensal (a partir do mês atual)
# =========================
def build_cartao_mensal_from_dividas(dividas_csv: str):
    df = pd.read_csv(dividas_csv, sep=";", dtype=str, keep_default_na=False)

    # Filtra ativas (se existir)
    if "Status da Dívida" in df.columns:
        df = df[df["Status da Dívida"].astype(str).str.strip().str.lower() == "ativa"].copy()

    # Normaliza colunas numéricas
    df["Nº de Parcelas"] = pd.to_numeric(df["Nº de Parcelas"], errors="coerce").fillna(0).astype(int)
    df["Parcelas Pagas"] = pd.to_numeric(df["Parcelas Pagas"], errors="coerce").fillna(0).astype(int)
    df["Valor da Parcela"] = df["Valor da Parcela"].apply(parse_number)

    df["parcelas_restantes"] = (df["Nº de Parcelas"] - df["Parcelas Pagas"]).clip(lower=0)

    start_m = month_start(date.today())

    rows = []
    for _, r in df.iterrows():
        n = int(r["parcelas_restantes"])
        if n <= 0:
            continue

        valor = float(r["Valor da Parcela"])
        for i in range(n):
            mes = add_months(start_m, i)
            rows.append({"mes": month_key(mes), "cartao_total": valor})

    if not rows:
        # sem dívidas
        cartao_m = pd.DataFrame({"mes": [month_key(start_m)], "cartao_total": [0.0]})
        end_m = start_m
    else:
        cartao_m = (
            pd.DataFrame(rows)
            .groupby("mes", as_index=False)["cartao_total"]
            .sum()
            .sort_values("mes")
        )
        # fim do horizonte é o último mês com pagamento
        end_m = datetime.strptime(cartao_m["mes"].iloc[-1] + "-01", "%Y-%m-%d").date()

    # Ajuste manual: fevereiro = 1089.08
    if MES_AJUSTE_CARTAO in set(cartao_m["mes"].tolist()):
        cartao_m.loc[cartao_m["mes"] == MES_AJUSTE_CARTAO, "cartao_total"] = float(VALOR_CARTAO_FEVEREIRO)
    else:
        # se por algum motivo fevereiro não existir no cronograma, cria a linha
        cartao_m = pd.concat([
            cartao_m,
            pd.DataFrame([{"mes": MES_AJUSTE_CARTAO, "cartao_total": float(VALOR_CARTAO_FEVEREIRO)}])
        ], ignore_index=True).sort_values("mes")

    return cartao_m, start_m, month_start(end_m)


# =========================
# 2) DESAFIO 52 SEMANAS: 52 semanas exatas -> soma por mês
#    - desconsiderar janeiro (mes=2026-01 => 0)
# =========================
def build_desafio_52s_mensal(data_inicio: str, base: float, inc: float, total_semanas: int):
    dt_inicio = datetime.strptime(data_inicio, "%d/%m/%Y").date()

    weeks = []
    cur = dt_inicio
    for n in range(1, total_semanas + 1):
        valor = base + (n - 1) * inc
        weeks.append({
            "mes": f"{cur.year:04d}-{cur.month:02d}",
            "valor_semana": float(valor)
        })
        cur = cur + relativedelta(days=7)

    dfw = pd.DataFrame(weeks)
    dfm = dfw.groupby("mes", as_index=False)["valor_semana"].sum().rename(columns={"valor_semana": "desafio_52s_total"})

    # Desconsiderar janeiro (no seu caso específico)
    dfm.loc[dfm["mes"] == "2026-01", "desafio_52s_total"] = 0.0

    return dfm


# =========================
# 3) UNIR (horizonte do cartão) e zerar desafio após 52 semanas automaticamente
# =========================
cartao_m, start_month, end_month = build_cartao_mensal_from_dividas(DIVIDAS_CSV)
desafio_m = build_desafio_52s_mensal(DATA_INICIO_52S, BASE_52S, INCREMENTO_52S, TOTAL_SEMANAS)

# Cria o range de meses do horizonte do cartão
months = [month_key(m) for m in build_month_range(start_month, end_month)]
df_final = pd.DataFrame({"mes": months})

# Merge
df_final = df_final.merge(cartao_m, on="mes", how="left")
df_final = df_final.merge(desafio_m, on="mes", how="left")

# Preenche vazios (desafio vira 0 depois que acabar, e também antes de começar)
df_final["cartao_total"] = df_final["cartao_total"].fillna(0.0)
df_final["desafio_52s_total"] = df_final["desafio_52s_total"].fillna(0.0)

# Salvar em CSV pt-BR friendly
df_final.to_csv(OUTPUT_CSV, index=False, sep=";", decimal=",")

print(f"✅ Gerado: {OUTPUT_CSV}")
print(f"📆 Horizonte: {df_final['mes'].iloc[0]} até {df_final['mes'].iloc[-1]}")
df_final.head(12)

✅ Gerado: data/tabela_unificada_cartao_desafio.csv
📆 Horizonte: 2026-02 até 2027-12


,mes,cartao_total,desafio_52s_total
0,2026-02,1089.08,70.0
1,2026-03,654.49,200.0
2,2026-04,563.90,250.0
3,2026-05,431.40,425.0
4,2026-06,431.40,430.0
5,2026-07,431.40,510.0
6,2026-08,379.41,750.0
7,2026-09,303.34,690.0
8,2026-10,256.35,770.0
9,2026-11,53.64,1075.0


In [8]:
# Arquivo que geramos antes
INPUT_CSV  = "data/tabela_unificada_cartao_desafio.csv"
OUTPUT_CSV = "data/tabela_unificada_com_fixos.csv"

# Regras de fixos
FIXO_PADRAO = 973.80
FIXO_MARCO_2026  = 1069.80
FIXO_A_PARTIR_JUN_2026 = 1013.80

MARCO_2026 = "2026-03"
JUNHO_2026 = "2026-06"


def fixos_por_mes(mes_yyyy_mm: str) -> float:
    """
    mes_yyyy_mm no formato 'YYYY-MM'
    Regras:
      - 2026-03 -> 1069.80
      - a partir de 2026-06 (inclusive) -> 1013.80 (vale para 2027+)
      - caso contrário -> 973.80
    """
    mes = str(mes_yyyy_mm).strip()

    # regra especial de março/2026
    if mes == MARCO_2026:
        return FIXO_MARCO_2026

    # regra "de junho/2026 em diante"
    # Comparação lexicográfica funciona para YYYY-MM
    if mes >= JUNHO_2026:
        return FIXO_A_PARTIR_JUN_2026

    return FIXO_PADRAO


# 1) Ler a tabela unificada (cartão + desafio)
df = pd.read_csv(INPUT_CSV, sep=";", decimal=",", dtype={"mes": str})

# 2) Garantir colunas numéricas
for c in ["cartao_total", "desafio_52s_total", "total_cartao_mais_desafio"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)

# 3) Aplicar fixos
df["fixos_total"] = df["mes"].apply(fixos_por_mes)

# (Opcional) conferir rapidamente 2027
print("📌 Fixos em 2027 (amostra):")
print(df[df["mes"].str.startswith("2027-")][["mes", "fixos_total"]].head(12))

# 4) Salvar
df.to_csv(OUTPUT_CSV, index=False, sep=";", decimal=",")
print(f"✅ Gerado: {OUTPUT_CSV}")

df.head(12)

📌 Fixos em 2027 (amostra):
        mes  fixos_total
11  2027-01       1013.8
12  2027-02       1013.8
13  2027-03       1013.8
14  2027-04       1013.8
15  2027-05       1013.8
16  2027-06       1013.8
17  2027-07       1013.8
18  2027-08       1013.8
19  2027-09       1013.8
20  2027-10       1013.8
21  2027-11       1013.8
22  2027-12       1013.8
✅ Gerado: data/tabela_unificada_com_fixos.csv


,mes,cartao_total,desafio_52s_total,fixos_total
0,2026-02,1089.08,70.0,973.8
1,2026-03,654.49,200.0,1069.8
2,2026-04,563.90,250.0,973.8
3,2026-05,431.40,425.0,973.8
4,2026-06,431.40,430.0,1013.8
5,2026-07,431.40,510.0,1013.8
6,2026-08,379.41,750.0,1013.8
7,2026-09,303.34,690.0,1013.8
8,2026-10,256.35,770.0,1013.8
9,2026-11,53.64,1075.0,1013.8


In [11]:
INPUT_CSV  = "data/tabela_unificada_com_fixos.csv"
OUTPUT_CSV = "data/tabela_final_com_salario.csv"

SALARIO_FEVEREIRO = 1400.00
SALARIO_PADRAO = 2000.00
MES_FEVEREIRO = "2026-02"


def salario_por_mes(mes: str) -> float:
    if mes == MES_FEVEREIRO:
        return SALARIO_FEVEREIRO
    return SALARIO_PADRAO


# Ler tabela
df = pd.read_csv(INPUT_CSV, sep=";", decimal=",", dtype={"mes": str})

# Garantir numéricos
for c in ["cartao_total", "desafio_52s_total", "fixos_total", "despesas_total"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)

# Aplicar salário
df["salario"] = df["mes"].apply(salario_por_mes)

# Calcular saldo
# df["saldo_mes"] = df["salario"] - df["despesas_total"]

# Saldo acumulado
# df["saldo_acumulado"] = df["saldo_mes"].cumsum()

# Salvar
df.to_csv(OUTPUT_CSV, index=False, sep=";", decimal=",")
print(f"✅ Gerado: {OUTPUT_CSV}")

df.head(12)

✅ Gerado: data/tabela_final_com_salario.csv


,mes,cartao_total,desafio_52s_total,fixos_total,salario
0,2026-02,1089.08,70.0,973.8,1400.0
1,2026-03,654.49,200.0,1069.8,2000.0
2,2026-04,563.90,250.0,973.8,2000.0
3,2026-05,431.40,425.0,973.8,2000.0
4,2026-06,431.40,430.0,1013.8,2000.0
5,2026-07,431.40,510.0,1013.8,2000.0
6,2026-08,379.41,750.0,1013.8,2000.0
7,2026-09,303.34,690.0,1013.8,2000.0
8,2026-10,256.35,770.0,1013.8,2000.0
9,2026-11,53.64,1075.0,1013.8,2000.0


In [14]:
INPUT_CSV = "data/extras.csv"  # ajuste se necessário
OUTPUT_EXTRAS = "data/extras_total.csv"

def parse_ptbr_number(x):
    if pd.isna(x):
        return 0.0
    if isinstance(x, (int, float, np.number)):
        return float(x)

    s = str(x).strip().replace("R$", "").replace(" ", "")
    if "," in s:
        s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except:
        return 0.0

# Ler arquivo
df = pd.read_csv(INPUT_CSV, sep=None, engine="python", dtype=str)

# Detectar colunas extras automaticamente
extra_cols = [c for c in df.columns if c.lower().startswith("extra_")]

# Converter para número
for c in extra_cols:
    df[c] = df[c].apply(parse_ptbr_number)

# Somar todas as colunas extras
df_extras = pd.DataFrame()
df_extras["mes"] = df["mes"]
df_extras["extras_total"] = df[extra_cols].sum(axis=1)

# Salvar CSV
df_extras.to_csv(OUTPUT_EXTRAS, index=False, sep=";", decimal=",")
print("✅ Gerado:", OUTPUT_EXTRAS)

df_extras.head()

✅ Gerado: data/extras_total.csv


,mes,extras_total
0,2026-02,326.40
1,2026-03,256.40
2,2026-04,280.12
3,2026-05,210.12
4,2026-06,280.12


In [18]:
MAIN_CSV = "data/tabela_final_com_salario.csv"
EXTRAS_CSV = "data/extras_total.csv"
OUTPUT_FINAL = "data/tabela_completa_final.csv"

# Ler tabelas
df_main = pd.read_csv(MAIN_CSV, sep=";", decimal=",", dtype={"mes": str})
df_extras = pd.read_csv(EXTRAS_CSV, sep=";", decimal=",", dtype={"mes": str})

# Garantir numérico
df_extras["extras_total"] = pd.to_numeric(df_extras["extras_total"], errors="coerce").fillna(0.0)

# Merge
df_final = df_main.merge(df_extras, on="mes", how="left")

# Se não houver extra no mês → 0
df_final["extras_total"] = df_final["extras_total"].fillna(0.0)

# Atualizar receita e saldo
df_final["receita_total"] = df_final["salario"] + df_final["extras_total"]
df_final["despesas_total"] = (
    df_final["fixos_total"] +
    df_final["cartao_total"] +
    df_final["desafio_52s_total"]
)

df_final["saldo_mes"] = df_final["receita_total"] - df_final["despesas_total"]
# df_final["saldo_acumulado"] = df_final["saldo_mes"].cumsum()

# Salvar
df_final.to_csv(OUTPUT_FINAL, index=False, sep=";", decimal=",")
print("✅ Gerado:", OUTPUT_FINAL)

df_final.head()

✅ Gerado: data/tabela_completa_final.csv


,mes,cartao_total,desafio_52s_total,fixos_total,salario,extras_total,receita_total,despesas_total,saldo_mes
0,2026-02,1089.08,70.0,973.8,1400.0,326.40,1726.40,2132.88,-406.48
1,2026-03,654.49,200.0,1069.8,2000.0,256.40,2256.40,1924.29,332.11
2,2026-04,563.90,250.0,973.8,2000.0,280.12,2280.12,1787.70,492.42
3,2026-05,431.40,425.0,973.8,2000.0,210.12,2210.12,1830.20,379.92
4,2026-06,431.40,430.0,1013.8,2000.0,280.12,2280.12,1875.20,404.92


# Geração do Relatório

## Importações

In [19]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from reportlab.lib.pagesizes import A4, landscape
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle,
    Image, PageBreak
)
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors
from reportlab.lib.units import cm

## Implementação

In [24]:
# =========================
# CONFIG
# =========================
INPUT_CSV = "data/tabela_completa_final.csv"
OUTPUT_PDF = "Relatorio_Gestao_Salarial_Mensal.pdf"

# salvar imagens temporárias dos gráficos
CHART_DIR = "charts_temp"
os.makedirs(CHART_DIR, exist_ok=True)

# =========================
# HELPERS
# =========================
def read_ptbr_csv(path: str) -> pd.DataFrame:
    # seu CSV está no formato pt-BR (separador ; e decimal ,)
    df = pd.read_csv(path, sep=";", decimal=",", dtype={"mes": str})
    return df

def to_float(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)
    return df

def fmt_money(x: float) -> str:
    # "R$ 1.234,56"
    s = f"{x:,.2f}"
    s = s.replace(",", "X").replace(".", ",").replace("X", ".")
    return f"R$ {s}"

def fmt_pct(x: float) -> str:
    return f"{x*100:.1f}%".replace(".", ",")

def safe_div(a, b):
    return a / b if b != 0 else 0.0

# =========================
# 1) LOAD + NORMALIZE
# =========================
df = read_ptbr_csv(INPUT_CSV)

# colunas esperadas (se faltar alguma, cria como 0)
expected = [
    "mes", "salario", "extras_total", "receita_total",
    "fixos_total", "cartao_total", "desafio_52s_total",
    "despesas_total", "saldo_mes", "saldo_acumulado"
]
for c in expected:
    if c not in df.columns:
        df[c] = 0.0

df = to_float(df, [
    "salario", "extras_total", "receita_total",
    "fixos_total", "cartao_total", "desafio_52s_total",
    "despesas_total", "saldo_mes", "saldo_acumulado"
])

# garante ordenação por mês (YYYY-MM ordena bem como string)
df = df.sort_values("mes").reset_index(drop=True)

# se receita_total não existir corretamente, recalcula
if (df["receita_total"].abs().sum() == 0) and ("salario" in df.columns):
    df["receita_total"] = df["salario"] + df.get("extras_total", 0.0)

# se despesas_total estiver zerada, recalcula
if df["despesas_total"].abs().sum() == 0:
    df["despesas_total"] = df["fixos_total"] + df["cartao_total"] + df["desafio_52s_total"]

# se saldo_mes zerado, recalcula
if df["saldo_mes"].abs().sum() == 0:
    df["saldo_mes"] = df["receita_total"] - df["despesas_total"]

# saldo acumulado
df["saldo_acumulado"] = df["saldo_mes"].cumsum()

# =========================
# 2) KPIs
# =========================
mes_ini = df["mes"].iloc[0]
mes_fim = df["mes"].iloc[-1]
n_meses = len(df)

total_receitas = df["receita_total"].sum()
total_despesas = df["despesas_total"].sum()
saldo_total = df["saldo_mes"].sum()

qtd_neg = int((df["saldo_mes"] < 0).sum())
qtd_pos = int((df["saldo_mes"] >= 0).sum())

pior = df.loc[df["saldo_mes"].idxmin()]
melhor = df.loc[df["saldo_mes"].idxmax()]

media_comprometimento = safe_div(total_despesas, total_receitas)  # % médio gasto da renda

# =========================
# 3) GRÁFICOS (salva PNG)
# =========================
# (A) Saldo mensal: barras verde/vermelho
plt.figure(figsize=(11, 4))
x = df["mes"].tolist()
y = df["saldo_mes"].tolist()
colors_bar = ["#1f9d55" if v >= 0 else "#d64545" for v in y]  # verde / vermelho
plt.bar(x, y, color=colors_bar)
plt.axhline(0, linewidth=1)
plt.title("Saldo mensal (Receita - Despesas)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
saldo_bar_path = os.path.join(CHART_DIR, "saldo_mensal.png")
plt.savefig(saldo_bar_path, dpi=200)
plt.close()

# (B) Saldo acumulado: linha
plt.figure(figsize=(11, 4))
plt.plot(x, df["saldo_acumulado"].tolist(), marker="o")
plt.axhline(0, linewidth=1)
plt.title("Saldo acumulado")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
saldo_acum_path = os.path.join(CHART_DIR, "saldo_acumulado.png")
plt.savefig(saldo_acum_path, dpi=200)
plt.close()

# (C) Composição das despesas: stacked (fixos + cartão + desafio)
plt.figure(figsize=(11, 4))
fixos = df["fixos_total"].tolist()
cartao = df["cartao_total"].tolist()
desafio = df["desafio_52s_total"].tolist()

plt.bar(x, fixos, label="Fixos")
plt.bar(x, cartao, bottom=fixos, label="Cartão")
bottom2 = [fixos[i] + cartao[i] for i in range(len(df))]
plt.bar(x, desafio, bottom=bottom2, label="Desafio 52s")
plt.title("Composição das despesas por mês")
plt.xticks(rotation=45, ha="right")
plt.legend()
plt.tight_layout()
stack_path = os.path.join(CHART_DIR, "despesas_stack.png")
plt.savefig(stack_path, dpi=200)
plt.close()

# =========================
# 4) PDF (ReportLab)
# =========================
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name="TitleBig", parent=styles["Title"], fontSize=20, spaceAfter=10))
styles.add(ParagraphStyle(name="H2", parent=styles["Heading2"], spaceBefore=12, spaceAfter=6))
styles.add(ParagraphStyle(name="Small", parent=styles["Normal"], fontSize=9, leading=11))
styles.add(ParagraphStyle(name="Muted", parent=styles["Normal"], fontSize=9, textColor=colors.grey))

doc = SimpleDocTemplate(
    OUTPUT_PDF,
    pagesize=landscape(A4),
    leftMargin=1.2*cm,
    rightMargin=1.2*cm,
    topMargin=1.2*cm,
    bottomMargin=1.2*cm,
)

story = []

# Capa / Header
story.append(Paragraph("Relatório Mensal — Gestão Salarial", styles["TitleBig"]))
story.append(Paragraph(f"Período: <b>{mes_ini}</b> até <b>{mes_fim}</b>  •  Total de meses: <b>{n_meses}</b>", styles["Muted"]))
story.append(Spacer(1, 10))

# Resumo executivo (tabela KPI)
story.append(Paragraph("Resumo executivo", styles["H2"]))

kpi_data = [
    ["Indicador", "Valor"],
    ["Total de receitas (salário + extras)", fmt_money(total_receitas)],
    ["Total de despesas (fixos + cartão + desafio)", fmt_money(total_despesas)],
    ["Saldo total do período", fmt_money(saldo_total)],
    ["Meses positivos", str(qtd_pos)],
    ["Meses negativos", str(qtd_neg)],
    ["Média de comprometimento (despesas/receitas)", fmt_pct(media_comprometimento)],
    ["Melhor mês (saldo)", f"{melhor['mes']} • {fmt_money(float(melhor['saldo_mes']))}"],
    ["Pior mês (saldo)", f"{pior['mes']} • {fmt_money(float(pior['saldo_mes']))}"],
]
kpi_table = Table(kpi_data, colWidths=[9*cm, 18*cm])
kpi_table.setStyle(TableStyle([
    ("BACKGROUND", (0,0), (-1,0), colors.HexColor("#111827")),
    ("TEXTCOLOR", (0,0), (-1,0), colors.white),
    ("FONTNAME", (0,0), (-1,0), "Helvetica-Bold"),
    ("FONTSIZE", (0,0), (-1,0), 10),
    ("ALIGN", (0,0), (-1,0), "LEFT"),
    ("GRID", (0,0), (-1,-1), 0.25, colors.lightgrey),
    ("ROWBACKGROUNDS", (0,1), (-1,-1), [colors.whitesmoke, colors.white]),
    ("VALIGN", (0,0), (-1,-1), "MIDDLE"),
    ("PADDING", (0,0), (-1,-1), 6),
]))
story.append(kpi_table)

# Aviso visual
story.append(Spacer(1, 10))
if qtd_neg > 0:
    story.append(Paragraph(
        f"⚠️ Atenção: você tem <b>{qtd_neg}</b> mês(es) com saldo negativo. "
        "Considere reduzir despesas variáveis ou antecipar quitação de parcelas para aliviar os próximos meses.",
        styles["Small"]
    ))
else:
    story.append(Paragraph(
        "✅ Ótimo: não há meses com saldo negativo no período projetado.",
        styles["Small"]
    ))

# Gráficos
story.append(Spacer(1, 12))
story.append(Paragraph("Gráficos", styles["H2"]))

img_w = 26*cm
img_h = 9*cm
story.append(Image(saldo_bar_path, width=img_w, height=img_h))
story.append(Spacer(1, 6))
story.append(Image(saldo_acum_path, width=img_w, height=img_h))
story.append(Spacer(1, 6))
story.append(Image(stack_path, width=img_w, height=img_h))

# Tabela detalhada
story.append(PageBreak())
story.append(Paragraph("Tabela detalhada (mês a mês)", styles["H2"]))
story.append(Paragraph("Saldo em verde = positivo | saldo em vermelho = negativo.", styles["Muted"]))
story.append(Spacer(1, 6))

# montar tabela detalhada
df_show = df.copy()
df_show["salario"] = df_show["salario"].apply(fmt_money)
df_show["extras_total"] = df_show["extras_total"].apply(fmt_money)
df_show["receita_total"] = df_show["receita_total"].apply(fmt_money)
df_show["fixos_total"] = df_show["fixos_total"].apply(fmt_money)
df_show["cartao_total"] = df_show["cartao_total"].apply(fmt_money)
df_show["desafio_52s_total"] = df_show["desafio_52s_total"].apply(fmt_money)
df_show["despesas_total"] = df_show["despesas_total"].apply(fmt_money)

# saldo com sinal visível
def fmt_money_signed(v):
    v = float(v)
    sign = "-" if v < 0 else ""
    return sign + fmt_money(abs(v)).replace("R$ ", "R$ ")

df_show["saldo_mes"] = df["saldo_mes"].apply(fmt_money_signed)
df_show["saldo_acumulado"] = df["saldo_acumulado"].apply(fmt_money_signed)

cols = ["mes","salario","extras_total","receita_total","fixos_total","cartao_total","desafio_52s_total","despesas_total","saldo_mes","saldo_acumulado"]
table_data = [cols] + df_show[cols].values.tolist()

col_widths = [2.2*cm, 3.0*cm, 3.0*cm, 3.2*cm, 3.0*cm, 3.0*cm, 3.0*cm, 3.2*cm, 3.2*cm, 3.4*cm]
det_table = Table(table_data, colWidths=col_widths, repeatRows=1)

# estilo base
style_cmds = [
    ("BACKGROUND", (0,0), (-1,0), colors.HexColor("#111827")),
    ("TEXTCOLOR", (0,0), (-1,0), colors.white),
    ("FONTNAME", (0,0), (-1,0), "Helvetica-Bold"),
    ("FONTSIZE", (0,0), (-1,0), 9),
    ("GRID", (0,0), (-1,-1), 0.25, colors.lightgrey),
    ("ROWBACKGROUNDS", (0,1), (-1,-1), [colors.whitesmoke, colors.white]),
    ("VALIGN", (0,0), (-1,-1), "MIDDLE"),
    ("ALIGN", (1,1), (-1,-1), "RIGHT"),
    ("ALIGN", (0,0), (0,-1), "LEFT"),
    ("PADDING", (0,0), (-1,-1), 4),
]

# colorir saldo_mes por linha
saldo_col_index = cols.index("saldo_mes")
for i in range(1, len(df)+1):
    v = float(df.loc[i-1, "saldo_mes"])
    if v < 0:
        style_cmds.append(("TEXTCOLOR", (saldo_col_index, i), (saldo_col_index, i), colors.HexColor("#d64545")))
        style_cmds.append(("FONTNAME", (saldo_col_index, i), (saldo_col_index, i), "Helvetica-Bold"))
    else:
        style_cmds.append(("TEXTCOLOR", (saldo_col_index, i), (saldo_col_index, i), colors.HexColor("#1f9d55")))
        style_cmds.append(("FONTNAME", (saldo_col_index, i), (saldo_col_index, i), "Helvetica-Bold"))

det_table.setStyle(TableStyle(style_cmds))
story.append(det_table)

# Gerar PDF
doc.build(story)

print("✅ PDF gerado:", OUTPUT_PDF)
print("📌 Gráficos salvos em:", CHART_DIR)

✅ PDF gerado: Relatorio_Gestao_Salarial_Mensal.pdf
📌 Gráficos salvos em: charts_temp
